
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# 2.2 LAB: Implementing Delta Sharing (Databricks-to-Databricks) \[Recipient]
## Lab Overview
In this hands-on lab, you will implement Delta Sharing as a data recipient. You'll work with a warehouse inventory management scenario where you are a **Regional Warehouse Manager** receiving shared data from **TechWare Labs** central inventory system.

**Your Role:** Data Recipient (Regional Warehouse Manager)

**Scenario:**
You manage a regional warehouse for TechWare Labs. The central inventory management team has offered to share live product and stock data with you through Delta Sharing. Instead of receiving periodic data dumps or building complex integrations, you'll mount their share directly into your workspace and access the data in real-time using standard SQL queries.

**Lab Objectives:**
By the end of this lab, you will be able to:
1. Obtain and share your metastore ID with data providers
2. Discover available providers and their shares
3. Mount a remote share to a local catalog
4. Explore and query shared data
5. Perform analytics on shared data
6. Understand the limitations and capabilities of shared data access

## Prerequisites

Before starting this lab, ensure that:
1. The provider has completed their lab and created the share
2. The provider has your metastore ID and has created a recipient for you
3. The provider has granted you access to the share

**Required Permissions:**
- `USE PROVIDER` privilege on the provider, OR
- Metastore admin role

## Setup

Run the following cell to set up your lab environment with configuration variables.

In [ ]:
%run ./Includes/Lab-Setup-2

## Exercise 1: Obtain Your Metastore ID (10 minutes)

Before you can receive shared data, you need to provide your metastore ID to the data provider. This unique identifier allows the provider to configure you as a recipient.

### Task 1.1: Get Your Metastore ID Using SQL
**TODO:** Query the system tables to retrieve your metastore's global ID.

In [ ]:
-- TODO: SELECT global_metastore_id FROM system.information_schema.metastores
SELECT global_metastore_id FROM system.information_schema.metastores

### Task 1.2: Alternative Method - Using DESCRIBE METASTORE
**TODO:** Use the DESCRIBE METASTORE command to view detailed metastore information.

**Look for:** The "Global Metastore ID" field in the output.

In [ ]:
-- TODO: DESCRIBE METASTORE
DESCRIBE METASTORE

**Action Required:** Copy your metastore ID from the output above and provide it to the data provider (your instructor or lab partner). The provider needs this to create a recipient for you.

**Note:** You can also obtain this ID through the UI:
1. Navigate to Catalog Explorer
2. Click the Delta Sharing (gear) icon
3. Click "Copy Sharing Identifier"

## Exercise 2: Discover Available Providers and Shares (10 minutes)

Once the provider has configured you as a recipient, you can discover what providers and shares are available to you.

### Task 2.1: List Available Providers
**TODO:** Display all providers that you have access to.

In [ ]:
-- TODO: SHOW PROVIDERS
SHOW PROVIDERS

### Task 2.2: Describe the Provider
**TODO:** Get detailed information about the TechWare Labs provider.

**Look for:** Region, cloud provider, and other metadata about the provider.

In [ ]:
-- TODO: DESCRIBE PROVIDER techware_labs
DESCRIBE PROVIDER techware_labs

### Task 2.3: List Available Shares from the Provider
**TODO:** Display all shares available from the TechWare Labs provider.

In [ ]:
-- TODO: SHOW SHARES IN PROVIDER techware_labs
SHOW SHARES IN PROVIDER techware_labs

**Question:** How many shares are available from TechWare Labs? What is the name of the share containing inventory data?

## Exercise 3: Mount the Share to a Local Catalog (15 minutes)

Now you'll mount the shared data to a catalog in your workspace. This creates a local reference to the remote share without copying any data.

### Task 3.1: Create a Catalog from the Share
**TODO:** Create a catalog named `shared_inventory_catalog` that mounts the `warehouse_inventory` share from the `techware_labs` provider.

**Syntax reminder:** The share reference should be in the format: `<provider_name>.<share_name>`

In [ ]:
-- TODO: CREATE CATALOG IF NOT EXISTS shared_inventory_catalog
-- TODO: USING SHARE <provider_name>.<share_name>
CREATE CATALOG IF NOT EXISTS shared_inventory_catalog
USING SHARE techware_labs.warehouse_inventory

### Task 3.2: Verify the Catalog Type
**TODO:** Describe the catalog you just created and verify its type.

**Look for:** The "Catalog Type" should be "DELTASHARING"

In [ ]:
-- TODO: DESCRIBE CATALOG shared_inventory_catalog
DESCRIBE CATALOG shared_inventory_catalog

## Exercise 4: Explore the Shared Data (15 minutes)

With the share mounted, let's explore the data structure and contents.

### Task 4.1: List Schemas in the Shared Catalog
**TODO:** Display all schemas available in the shared catalog.

In [ ]:
-- TODO: SHOW SCHEMAS IN shared_inventory_catalog
SHOW SCHEMAS IN shared_inventory_catalog

### Task 4.2: List Tables in the Inventory Schema
**TODO:** Display all tables in the `inventory` schema.

In [ ]:
-- TODO: SHOW TABLES IN shared_inventory_catalog.inventory
SHOW TABLES IN shared_inventory_catalog.inventory

**Question:** How many tables are shared? What are their names?

### Task 4.3: Examine the Products Table Structure
**TODO:** Describe the structure of the products table to understand its schema.

In [ ]:
-- TODO: DESCRIBE TABLE shared_inventory_catalog.inventory.products
DESCRIBE TABLE shared_inventory_catalog.inventory.products

## Exercise 5: Query and Analyze Shared Data (20 minutes)

Now let's query the shared data and perform some analysis relevant to warehouse management.

### Task 5.1: View All Products
**TODO:** Query all records from the products table.

In [ ]:
-- TODO: SELECT * FROM shared_inventory_catalog.inventory.products
SELECT * FROM shared_inventory_catalog.inventory.products

### Task 5.2: View Current Warehouse Stock
**TODO:** Query all records from the warehouse_stock table.

In [ ]:
-- TODO: SELECT * FROM shared_inventory_catalog.inventory.warehouse_stock
SELECT * FROM shared_inventory_catalog.inventory.warehouse_stock

### Task 5.3: Join Products with Stock Levels
**TODO:** Write a query that joins the products and warehouse_stock tables to show product details along with stock levels.

**Include these columns:**
- product_name
- category
- unit_price
- warehouse_location
- quantity_on_hand
- reorder_level (from products table)

**Order by:** warehouse_location and product_name

In [ ]:
-- TODO: Write a SELECT statement joining products (p) and warehouse_stock (w)
-- TODO: ON p.product_id = w.product_id
-- TODO: SELECT the required columns
-- TODO: ORDER BY warehouse_location, product_name
SELECT 
  p.product_name,
  p.category,
  p.unit_price,
  w.warehouse_location,
  w.quantity_on_hand,
  p.reorder_level
FROM shared_inventory_catalog.inventory.products p
JOIN shared_inventory_catalog.inventory.warehouse_stock w
  ON p.product_id = w.product_id
ORDER BY w.warehouse_location, p.product_name

### Task 5.4: Identify Low Stock Items
**TODO:** Write a query to identify products that are below their reorder level at any warehouse.

**Requirements:**
- Join products and warehouse_stock tables
- Filter WHERE quantity_on_hand < reorder_level
- Show product_name, warehouse_location, quantity_on_hand, reorder_level
- Calculate how many units below reorder level (reorder_level - quantity_on_hand) as `units_needed`
- Order by units_needed DESC

In [ ]:
-- TODO: Write query to find products below reorder level
SELECT 
  p.product_name,
  w.warehouse_location,
  w.quantity_on_hand,
  p.reorder_level,
  (p.reorder_level - w.quantity_on_hand) AS units_needed
FROM shared_inventory_catalog.inventory.products p
JOIN shared_inventory_catalog.inventory.warehouse_stock w
  ON p.product_id = w.product_id
WHERE w.quantity_on_hand < p.reorder_level
ORDER BY units_needed DESC

**Question:** Which products need immediate reordering? Which warehouse has the most critical stock shortage?

### Task 5.5: Calculate Total Inventory Value by Warehouse
**TODO:** Write a query to calculate the total inventory value at each warehouse.

**Requirements:**
- Join products and warehouse_stock
- Calculate: quantity_on_hand * unit_price for each item
- GROUP BY warehouse_location
- Show warehouse_location and total_inventory_value
- Format the value to 2 decimal places
- Order by total_inventory_value DESC

In [ ]:
-- TODO: Write query to calculate total inventory value by warehouse
SELECT 
  w.warehouse_location,
  ROUND(SUM(w.quantity_on_hand * p.unit_price), 2) AS total_inventory_value
FROM shared_inventory_catalog.inventory.products p
JOIN shared_inventory_catalog.inventory.warehouse_stock w
  ON p.product_id = w.product_id
GROUP BY w.warehouse_location
ORDER BY total_inventory_value DESC

### Task 5.6: Analyze Stock by Category
**TODO:** Write a query to show total quantity and number of different products by category across all warehouses.

**Requirements:**
- Join products and warehouse_stock
- GROUP BY category
- Calculate SUM(quantity_on_hand) as total_units
- Calculate COUNT(DISTINCT product_id) as product_count
- Order by total_units DESC

In [ ]:
-- TODO: Write query to analyze stock by category
SELECT 
  p.category,
  SUM(w.quantity_on_hand) AS total_units,
  COUNT(DISTINCT p.product_id) AS product_count
FROM shared_inventory_catalog.inventory.products p
JOIN shared_inventory_catalog.inventory.warehouse_stock w
  ON p.product_id = w.product_id
GROUP BY p.category
ORDER BY total_units DESC

## Exercise 6: Understand Read-Only Limitations (5 minutes)

Delta Sharing provides read-only access to shared data. Let's verify this.

### Task 6.1: Attempt to Modify Shared Data
**TODO:** Try to delete a record from the shared products table.

**Expected Result:** This should fail with an error indicating the table is read-only.

**Note:** The DELETE statement below is commented out. Uncomment it to test.

In [ ]:
-- TODO: Uncomment the line below and run to see the read-only error
-- DELETE FROM shared_inventory_catalog.inventory.products WHERE product_id = 1001;

**Question:** Why is read-only access beneficial for data sharing scenarios?

## Exercise 7: (Optional) Create Views on Shared Data (10 minutes)

While you can't modify shared data, you can create views that reference it in your own catalogs.

### Task 7.1: Create a View for Low Stock Alert
**TODO:** Create a view in your default catalog that shows products below reorder level.

**Note:** You'll need to use your own catalog (not the shared catalog) to create the view. Use the catalog assigned to your workspace.

In [ ]:
-- TODO: First, create a schema in your default catalog if needed
-- CREATE SCHEMA IF NOT EXISTS <your_catalog>.warehouse_analysis;

-- TODO: Create a view that references the shared data
-- CREATE OR REPLACE VIEW <your_catalog>.warehouse_analysis.low_stock_alert AS
-- SELECT 
--   p.product_name,
--   w.warehouse_location,
--   w.quantity_on_hand,
--   p.reorder_level,
--   (p.reorder_level - w.quantity_on_hand) AS units_needed
-- FROM shared_inventory_catalog.inventory.products p
-- JOIN shared_inventory_catalog.inventory.warehouse_stock w
--   ON p.product_id = w.product_id
-- WHERE w.quantity_on_hand < p.reorder_level;

## Lab Cleanup

When you're finished with the lab, you can clean up the mounted catalog.

**Note:** This only removes your local mount point - it doesn't affect the provider's data or other recipients' access.

In [ ]:
-- Uncomment to drop the mounted catalog when finished
-- DROP CATALOG IF EXISTS shared_inventory_catalog;

## Summary

Congratulations! You've successfully completed the recipient lab. You:

✅ Obtained and shared your metastore ID with the provider
✅ Discovered available providers and shares
✅ Mounted a remote share to a local catalog
✅ Explored the structure of shared data
✅ Performed analytical queries on shared data
✅ Joined multiple shared tables
✅ Created business insights from shared inventory data
✅ Understood read-only access limitations

**Key Takeaways:**

1. **Zero Data Movement**: You accessed live data from the provider without any data copying
2. **Real-Time Access**: Any updates the provider makes are immediately available to you
3. **Standard SQL**: You used familiar SQL syntax - Delta Sharing is transparent to analysts
4. **Read-Only Security**: The provider maintains control while you get safe, read-only access
5. **No Storage Costs**: You pay no storage costs for shared data

**Real-World Applications:**
- Supply chain partners sharing inventory and logistics data
- Financial institutions sharing market data
- Healthcare organizations sharing anonymized patient data
- Retail chains sharing sales data with vendors
- Government agencies sharing public datasets

**Next Steps:**
In the next module, you'll learn about:
- Sharing data with non-Databricks recipients
- Advanced security features (row-level, column-level filtering)
- Sharing views and materialized views
- Monitoring and governance best practices

---
&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>